## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [61]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette, PneumaticSampleLoader
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
sys.path.append('../..')
import stober_synthesis_utils as stober

In [4]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = '2025_03_11_mesoporoustest.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [5]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

0
gcode cmd:  M409 K"move.axes[].homed"
response in connect:  [True, True, True, True]
gcode cmd:  M409 K"move.axes[]"
gcode cmd:  T
gcode cmd:  M409 K"tools"
gcode cmd:  M409 K"move.axes"
gcode cmd:  G90
gcode cmd:  G90


In [7]:
sample_table = pd.read_csv('2025_03_11_mesoporous_sampletable - Sheet1.csv')
#sample_table_2 = pd.read_csv('SampleTable_ReproReplicates_10000.csv')


In [8]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [9]:
sample_table['well'] = ['A1', 'A2', 'A3', 'A4', 'B1']

In [10]:
sample_table

,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,f127_voume,well
0,1,371,733,1420,2590,2190,2700,A1
1,2,371,733,1420,2590,2190,2700,A2
2,3,371,733,1420,2590,2190,2700,A3
3,4,371,733,1420,2590,2190,2700,A4
4,5,371,733,1420,2590,2190,2700,B1


In [143]:
jubilee.park_tool()

gcode cmd:  M114
gcode cmd:  T-1


In [11]:
sample_table['water_volume'].sum()

7100

## Load Tools

In [94]:
syringe_10 = syringe.HTTPSyringe.from_config(2, "../../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(1, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_Hamiltonsyringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_Hamiltonsyringe.json")
syringe_1_4 = syringe.HTTPSyringe.from_config(0, '../../../science-jubilee/src/science_jubilee/tools/configs/1cc_4_syringe.json')

Syringe name:  10cc_1
{'capacity': 10000, 'empty_position': 1830, 'full_position': 1217, 'name': '10cc_1'}
Syringe name:  1cc_1
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_1'}
Syringe name:  1cc_2_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_2_hamilton'}
Syringe name:  1cc_3_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_3_hamilton'}
Syringe name:  1cc_4
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_4'}


In [96]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)
jubilee.load_tool(syringe_1_4)

gcode cmd:  M409 K"tools"
gcode cmd:  M409 K"tools"
gcode cmd:  M409 K"tools"
gcode cmd:  M409 K"tools"
gcode cmd:  M409 K"tools"


In [97]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3
surfactant_syringe = syringe_1_4

## Load water syringe with water

In [16]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [144]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [24]:
water_syringe.set_pulsewidth(water_syringe.full_position+1, s = 150)

In [25]:
water_syringe.set_pulsewidth(water_syringe.full_position+250, s = 500)

In [98]:
water_syringe.load_syringe(590, water_syringe.full_position+250)

Loaded syringe, remaining volume 590 uL


## Load ammonia syringe

In [26]:
#ammonia_syringe.set_pulsewidth(1420, s = 10)

In [37]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [145]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [45]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 200)

In [46]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+250, s = 500)

In [99]:
ammonia_syringe.load_syringe(560, ammonia_syringe.full_position+250)

Loaded syringe, remaining volume 560 uL


## Load TEOS syringe

In [48]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [146]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [56]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+1, s = 200)

In [57]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+300, s = 200)

In [100]:
teos_syringe.load_syringe(480, teos_syringe.full_position+300)

Loaded syringe, remaining volume 480 uL


## load ethanol syringe

In [51]:
#jubilee.park_tool()

In [26]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


In [33]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


In [34]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+1, s = 1000)

In [35]:
#mix_syringe.empty_position

In [36]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+250, s = 500)

In [101]:
mix_syringe.load_syringe(6100, mix_syringe.full_position+250)

Loaded syringe, remaining volume 6100 uL


## Don't need to load surfactant syringe

In [102]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


## Load AFL

In [135]:
safe_pos = (5, 200, 100)
cell_pos = (217.5, 69.0, 61.5)
url = 'http://192.168.1.4'
port = '5000'
name = 'PSL'
username = 'test'
password = 'domo_arigato'

In [136]:
psl = PneumaticSampleLoader.PneumaticSampleLoader(url, port, name, cell_pos, safe_pos, username, password)

status r code 200
status:  b'["State: READY","Arm State: UP","Rinse 1 tank: 950 mL","Rinse 2 tank: 950 mL","Waste tank: 0 mL","Relay status: {\'arm-up\': True, \'arm-down\': False, \'rinse1\': False, \'rinse2\': False, \'blow\': False, \'piston-vent\': True, \'postsample\': False}","Not Rinsing","No action yet..."]\n'


In [137]:
jubilee.load_tool(psl)

## Load labware


In [63]:
samples = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 2)
samples.manual_offset([(31.1, 169.0), (121.1, 169.0), (121.1, 113.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [64]:
stocks_main = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_main.manual_offset([(30.5, 266.0),  (120.5, 266.0), (120.5, 210.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [82]:
teos_stocks = [stocks_main[0]]
ammonia_stocks = [stocks_main[1]]
water_stocks = [stocks_main[2]]
ethanol_stocks = [stocks_main[5]]
ctab_stocks = [stocks_main[3]]
f127_stocks = [stocks_main[4]]
rinse_stocks_apdist = [stocks_main[9], stocks_main[10], stocks_main[11]]

## Check sample alignment

In [47]:
jubilee.pickup_tool(mix_syringe)
for i, row in sample_table.iterrows():
    loc = samples[row['well']]
    jubilee.move_to(x = loc.x, y = loc.y)
    print(loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = row['well']
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

169.0


Is the tip centered over sample A1? 


169.0


Is the tip centered over sample A2? 


169.0


...

Is the tip centered over sample B4? 


113.0


Is the tip centered over sample C1? 


113.0


Is the tip centered over sample C2? 


In [49]:
sample_table

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,well
0,0,82c085e1-a523-40ce-9e89-4fa0bf546525,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B2
1,1,747534b1-7370-49a5-bdc1-4fce5227f391,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B3
2,2,e5f6e82f-0282-46c8-be77-d8914adca9b5,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B4
3,3,c87ac840-ddde-43ab-a8e3-b5e3eb5a273d,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,C1
4,4,d034d82c-e329-484c-91cc-75ea655a1446,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,C2


In [57]:
jubilee.park_tool()


In [106]:
loc = samples['A8']
jubilee.move_to(x = loc.x, y = loc.y)
print(loc.y)
jubilee.move_to(z = loc.top_+7)

175.8614659806774


# Experiment


In [69]:
location_lookup = {row['uuid']:samples[row['well']] for i, row in sample_table.iterrows()}


In [70]:
sample_table['ethanol_volume'].sum()

12950

In [71]:
sample_table.iloc[-1:]

,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,f127_voume,well
4,5,371,733,1420,2590,2190,2700,B1


In [72]:
ethanol_stocks_apdist[-1:]

[Well B4 form 20mLscintillation 12 Well Plate 18000 µL on slot 4]

In [73]:
sample_number = 0

In [78]:
sample_table_select = sample_table.iloc[sample_number:sample_number+1]

In [79]:
sample_table_select

,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,f127_voume,well
0,1,371,733,1420,2590,2190,2700,A1


In [84]:
ethanol_stocks

[Well B2 form 20mLscintillation 12 Well Plate 18000 µL on slot 4]

In [88]:
ethanol_stocks

[Well B2 form 20mLscintillation 12 Well Plate 18000 µL on slot 4]

## add ethanol

In [103]:
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table_select, location_lookup, 'ethanol_volume', ethanol_stocks, stocks_usable_volume=15000, dwell_time = 10)

gcode cmd:  M114
gcode cmd:  T2
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0  X31.10 Y169.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  T-1


In [104]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)

In [106]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


## Dispense water


In [107]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_select, location_lookup, 'water_volume', water_stocks, dwell_time = 7)

gcode cmd:  M114
gcode cmd:  T1
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0  X90.50 Y266.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X90.50 Y266.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X31.10 Y169.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X90.50 Y266.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd: 

## dispense Ammonia

In [108]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_select, location_lookup, 'ammonia_volume', ammonia_stocks, refill_dwell = 10)

gcode cmd:  M114
gcode cmd:  T3
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0  X60.50 Y266.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X60.50 Y266.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X31.10 Y169.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  T-1


## dispense F127

In [112]:
stober.add_reactants_batch(jubilee, surfactant_syringe, mix_syringe, sample_table_select, location_lookup, 'f127_voume', f127_stocks, refill_dwell = 10)

gcode cmd:  M114
gcode cmd:  T0
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0  X30.50 Y238.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X30.50 Y238.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X31.10 Y169.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X30.50 Y238.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd: 

In [111]:
sample_table

,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,f127_voume,well
0,1,371,733,1420,2590,2190,2700,A1
1,2,371,733,1420,2590,2190,2700,A2
2,3,371,733,1420,2590,2190,2700,A3
3,4,371,733,1420,2590,2190,2700,A4
4,5,371,733,1420,2590,2190,2700,B1


In [119]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)
jubilee.pickup_tool(surfactant_syringe)
for stock in rinse_stocks_apdist[2:]:
    surfactant_syringe.mix(900, 2, stock)
jubilee.park_tool()

Loaded syringe, remaining volume 0 uL
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  T0
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0  X120.50 Y210.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z7.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400


TypeError: park_tool() takes 1 positional argument but 2 were given

In [120]:
jubilee.park_tool()

gcode cmd:  M114
gcode cmd:  T-1


## Dispense CTAB

In [122]:
stober.add_reactants_batch(jubilee, surfactant_syringe, mix_syringe, sample_table_select, location_lookup, 'ctab_volume', ctab_stocks, refill_dwell = 10)

gcode cmd:  M114
gcode cmd:  T0
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0  X120.50 Y266.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X120.50 Y266.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X31.10 Y169.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X120.50 Y266.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cm

In [123]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)
jubilee.pickup_tool(surfactant_syringe)
for stock in rinse_stocks_apdist:
    surfactant_syringe.mix(900, 2, stock)
jubilee.park_tool()

Loaded syringe, remaining volume 0 uL
gcode cmd:  M114
gcode cmd:  T0
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0  X60.50 Y210.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z7.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X90.50 Y210.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z7.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X120.50 Y210.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z7.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  T-1


## mix precursors

In [124]:
stober.first_mix(jubilee, mix_syringe, 5000, location_lookup, rinse_stocks_apdist, 5)

gcode cmd:  M114
gcode cmd:  T2
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0  X31.10 Y169.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X60.50 Y210.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z17.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X90.50 Y210.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z17.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X120.50 Y210.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z17.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:

KeyboardInterrupt: 

In [125]:
jubilee.park_tool()

gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  T-1


## Add TEOS and mix

In [126]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_select, location_lookup, 'teos_volume', teos_stocks, mix_after=(8000, 5, rinse_stocks_apdist), wait = False)

gcode cmd:  M114
gcode cmd:  T4
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0  X31.10 Y169.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  T-1
gcode cmd:  M114
gcode cmd:  T2
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0  X31.10 Y169.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z12.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X60.50 Y210.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z17.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 Z80.50     F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0  X90.50 Y210.00   F6000.00 
gcode cmd:  M400
gcode cmd:  G90
gcode cmd:  G0 Z17.50     F6000.00 
gcode cmd:  M400
gcode cmd:  M114
gcode cmd:  G90
gcode cmd:  G0 

In [ ]:
time.sleep(5*60)

## Load cell

In [ ]:
psl.load_sample(mix_syringe, samples['A1'], 1000)

In [134]:
cell_pos

(200, 50, 20)

In [138]:
jubilee.move_to(x = cell_pos[0], y = cell_pos[1])

gcode cmd:  G90
gcode cmd:  G0  X217.50 Y69.00   F6000.00 
gcode cmd:  M400
